In [ ]:
import requests
import pandas as pd
import time

APP_ID = "5b720389"
API_KEY = "0a74f60c5c4ebeed8ed3c975f1844fd3"
url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
headers = {
    "x-app-id": APP_ID,
    "x-app-key": API_KEY,
    "Content-Type": "application/json"
}

# Example: generate 200+ food names by combining lists of common foods
fruits = ["apple", "banana", "orange", "grape", "pear", "peach", "mango", "kiwi", "pineapple", "strawberry"]
veggies = ["carrot", "broccoli", "spinach", "cabbage", "tomato", "onion", "pepper", "cucumber", "celery", "lettuce"]
proteins = ["chicken breast", "ground beef", "salmon", "egg", "tofu", "pork chop", "shrimp", "tuna", "turkey", "lamb"]
grains = ["white rice", "brown rice", "oatmeal", "quinoa", "pasta", "bread", "bagel", "cereal", "granola", "tortilla"]
snacks = ["peanut butter", "almonds", "cashews", "popcorn", "potato chips", "chocolate", "cookie", "ice cream", "pizza", "hamburger"]

# Combine and repeat to get 200+ items
base_list = fruits + veggies + proteins + grains + snacks
food_queries = (base_list * ((200 // len(base_list)) + 1))[:200]

results = []

for idx, query in enumerate(food_queries, 1):
    print(f"Processing {idx}/{len(food_queries)}: {query}")
    data = {"query": query, "timezone": "US/Eastern"}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        for food in response.json().get("foods", []):
            results.append({
                "food_name": food.get("food_name"),
                "serving_qty": food.get("serving_qty"),
                "serving_unit": food.get("serving_unit"),
                "calories": food.get("nf_calories"),
                "protein": food.get("nf_protein"),
                "fat": food.get("nf_total_fat"),
                "carbohydrates": food.get("nf_total_carbohydrate"),
                "sugar": food.get("nf_sugars"),
            })
    else:
        print(f"Error {response.status_code} for query '{query}': {response.text}")
    time.sleep(0.5)

df = pd.DataFrame(results)
df.to_csv("nutritionix_bulk_data.csv", index=False)
print(f"Saved {len(df)} records to nutritionix_bulk_data.csv")
